In [1]:
import os
import pandas as pd
import datetime as dt
import requests
from bs4 import BeautifulSoup as bs
import json
import datetime as dt
import time

In [2]:
##################### DO NOT TOUCH THIS FUNCTION LIBERALLY ###########################

def pull_igtv(show_alias):
    
    ## Site URL initialization
    site = 'https://www.instagram.com/{0}/channel/?hl=en'.format(show_alias)

    ## Requesting HTML content and storing in soup instance
    soup = bs(requests.get(site).text, 'lxml')

    ## Pulling overview of show, i.e., number of posts, followers, and following
    show_overview_info_metas = [str(meta) for meta in soup.find_all('meta')]
    filtered_metas = list(filter(lambda meta: 'Follow' in meta, show_overview_info_metas))[0].split('"')[1].split(' - ')[0]
    show_overview_summary = filtered_metas

    ## Extracting json element containing all post info
    raw_dict_string = str(soup.find_all('body')[0].find('script')).split('_sharedData = ')[1].replace(';</script>','')
    raw_json = json.loads(raw_dict_string)
    main_dict = raw_json['entry_data']['ProfilePage'][0]['graphql']['user']['edge_felix_video_timeline']
    
    ## Extracting number of posts from igtv show
    igtv_post_count = main_dict['count']
    
    ## Creating show data summary data frame by iterating
    ## over all posts and extracting necessary information
    igtv_dict = {'product_type':[],
                 'title':[],
                 'id':[],
                 'shortcode': [],
                 'owner_id': [],
                 'owner_username': [],
                 'snapshot_date_unix': [],
                 'video_duration_seconds': [],
                 'count_of_comments': [],
                 'number_of_likes': [],
                 'video_view_count': []}

    for n in main_dict['edges']:
        node = n['node']
        igtv_dict['product_type'].append(node['product_type'])
        igtv_dict['title'].append(node['title'])
        igtv_dict['id'].append(node['id'])
        igtv_dict['shortcode'].append(node['shortcode'])
        igtv_dict['owner_id'].append(node['owner']['id'])
        igtv_dict['owner_username'].append(node['owner']['username'])
        igtv_dict['snapshot_date_unix'].append(node['taken_at_timestamp'])
        igtv_dict['video_duration_seconds'].append(node['video_duration'])
        igtv_dict['count_of_comments'].append(node['edge_media_to_comment']['count'])
        igtv_dict['number_of_likes'].append(node['edge_liked_by']['count'])
        igtv_dict['video_view_count'].append(node['video_view_count'])

    show_data_df = pd.DataFrame(igtv_dict)
    
    show_data_df['igtv_post_count'] = igtv_post_count
    show_data_df['show_overview_summary'] = show_overview_summary
    
    return show_data_df

In [5]:
##################### ADD SHOW NAMES TO LIST HERE #########################

show_aliases = ['fallontonight', 'agt']

#################### LOOP TO CREATE DATAFRAME OF ALL DATA ##############################

all_shows_df = []

for i, show in enumerate(show_aliases):
    curr_show_data_df = pull_igtv(show)
    all_shows_df.append(curr_show_data_df)
    print("Current Show: {0} --- Completed {1}%".format(show, round(i/len(show_aliases)*100,2)))
    time.sleep(60)

print("Completed {0}%".format(round(i/len(show_aliases)*100,2)))

all_shows_df = pd.concat(all_shows_df, axis = 0).reset_index(drop = True)

currdate = dt.datetime.today().strftime('%Y-%m-%d')

################### SAVE TO CSV ############################
all_shows_df.to_csv('igtv_extract_{}.csv'.format(currdate))

Current Show: fallontonight --- Completed 0.0%
Current Show: agt --- Completed 50.0%


## Dictionary codes for personal reference

product_type - product_type <br>
title - title <br>
id - id <br>
shortcode - shortcode <br>
owner - id <br>
owner - username <br>
taken_at_timestamp - unix time <br>
video_duration - in seconds <br>
edge_media_to_comment -- comments count <br>
edge_liked_by -- number of likes <br>
video_view_count - video_view_count <br>

